In [1]:
import pandas as pd
import json
import itertools

In [2]:
full_data = pd.read_csv("dataset/movies_metadata.csv")

c:\users\oro_s\documents\4- code\python\tests\venv\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data = full_data.loc[:, ["title", "genres", "overview"]]

In [4]:
data.head()

,title,genres,overview
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...


In [5]:
# cast genre col to python objects
genres_col = list(data["genres"].apply(lambda x: [q["name"] for q in eval(x)]).values)
# flatten into a single list
genre_occurances = list(itertools.chain.from_iterable(genres_col))
genres = set(genre_occurances)
# keep genres that appear more than 1 time
common_genres_counts = {g: genre_occurances.count(g) for g in genres if genre_occurances.count(g) > 1}
common_genres = list(common_genres_counts.keys())


In [6]:
print("{} movies left".format(len(data)))

45466 movies left


In [7]:
# now let us create the targets by removing non used genres
data["targets"] = [[g for g in l if g in common_genres] for l in genres_col]
# drop genres column
data = data.drop("genres", axis=1)
# remove movies without any genre
data = data[data['targets'].map(lambda d: len(d)) > 0]
print("{} movies left".format(len(data)))

43021 movies left


In [8]:
# drop movies with some non defined value
data = data.dropna(axis=0)
print("{} movies left".format(len(data)))

42321 movies left


In [9]:
data.head(1000)

,title,overview,targets
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]"
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]"
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]"
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy]
5,Heat,"Obsessive master thief, Neil McCauley leads a ...","[Action, Crime, Drama, Thriller]"
6,Sabrina,An ugly duckling having undergone a remarkable...,"[Comedy, Romance]"
7,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...","[Action, Adventure, Drama, Family]"
8,Sudden Death,International action superstar Jean Claude Van...,"[Action, Adventure, Thriller]"
9,GoldenEye,James Bond must unmask the mysterious head of ...,"[Adventure, Action, Thriller]"
